## imports

In [33]:
import pandas as pd
import numpy as np
import cv2
import SimpleITK as sitk
import os
import shutil

## BrEaST Data Preprocessing into images and labels structure

In [26]:
src_dir = '../data/mass_data/BrEaST-Lesions_USG-images_and_masks/'
lab_dir = '../data/mass_data/BrEaST-Lesions_USG-images_and_masks/labels/'
img_dir = '../data/mass_data/BrEaST-Lesions_USG-images_and_masks/images/'

In [31]:
for file in os.listdir(path = src_dir):
    tum_det = file.split('_')
    if len(tum_det) > 1  and tum_det[1] == 'tumor.png':
        if not os.path.exists(lab_dir + file):
            shutil.copy(src_dir + file, lab_dir + file)

In [32]:
for file in os.listdir(path = src_dir):
    tum_det = file.split('_')
    if len(tum_det) < 2 and file.lower().endswith('.png'):
        if not os.path.exists(img_dir + file):
            shutil.copy(src_dir + file, img_dir + file)